In [ ]:
import firebase_admin
from firebase_admin import firestore
import os


In [ ]:
cred = firebase_admin.credentials.Certificate('key.json')
default_app = firebase_admin.initialize_app(cred)
db = firestore.client()
instructions_ref = db.collection('instructions')


In [ ]:
list_instruction = list(instructions_ref.get())
len(list_instruction)


In [ ]:
y = [x.to_dict()['work_id'] for x in list_instruction if 'Manhattan' in x.to_dict()['region'] and x.to_dict()['work_id'] not in ['A1198W1SPF1R4','APRZ7BR8C0ZMQ','A21LONLNBOB8Q', 'AY7WPVKHVNBLG', 'A320QA9HJFUOZO','A2DDPSXH2X96RF','A2GYGCLTMIGDJT','A3QAL3X23LHJRV','ALYR5CI2SM2JC']]
len(y)

In [ ]:
import pandas as pd
qualified_work_list = pd.read_csv("/home/onlp_gcp_biu/cabby/cabby/geo/visualization/qualified_workers.csv", header=None)[0].tolist()

qualified_work_list

In [ ]:
for i in set(y):
  if i not in qualified_work_list:
    print (i) 

In [ ]:


instruction_data = [
  e.to_dict()['key'] for e in list_instruction if (
    'X' == e.to_dict()['work_id']) ]

for id in instruction_data:
  instructions_ref.document(id).update(
    {'review': 'RVS_excellent'})


In [ ]:
import json

instruction_dir = "/mnt/hackney/human_data"

def save_split(split_name, split_set):

  split_path = os.path.join(instruction_dir, split_name)
  if os.path.exists(split_path):
    os.remove(split_path)
  with open(split_path, 'a') as outfile:
    for sample in split_set:
      json.dump(sample, outfile, default=lambda o: o.__dict__)
      outfile.write('\n')
      outfile.flush()

  print (f"Size of {split_name} set:{len(split_set)} saved to: {split_path}")


# Save set splits

dev_ids = ['X']
test_ids = ['Y']


test_set =  [
  x.to_dict() for x in list_instruction if 'Manhattan' in x.to_dict()['region'] and x.to_dict()['work_id'] in test_ids]

save_split("test.json", test_set)

dev_set =  [
  x.to_dict() for x in list_instruction if 'Manhattan' in x.to_dict()['region'] and x.to_dict()['work_id'] in dev_ids]

save_split("dev.json", dev_set)

train_set = [
  x.to_dict() for x in list_instruction if 'Manhattan' in x.to_dict()['region'] and x.to_dict()['work_id'] not in dev_ids+test_ids]

save_split("train.json", train_set)
